# Checking if a Product offering exists in the Warehouse or not

# Query Flow -> New Demand - Product Offering : 10 units say, 1st Check - Warehouse (Warehouse To PO) - if demand unsatisfied, 2nd Check - Facility (Facility To PO), 3rd Check - Parts (Parts To Facility), 4th Check - Warehouse (Warehouse To Parts), 5th Check - Supplier (Supplier To Warehouse)

# Experiment Optimization

In [ ]:
#Experiment optimization -- 
import json

# Function to query warehouses connected to a product offering
def query_warehouses(product_offering_id, file_path):

    # Load the data from the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)
   
    warehouses = []
    link_dict = {1:"WAREHOUSEToPRODUCT_OFFERING", 2 : "FACILITYToPRODUCT_OFFERING"}
    node_dict = {1:"WAREHOUSE", 2 : "FACILITY"} 
    
    # Iterate through link values to find WAREHOUSEToPRODUCT_OFFERING relationships
    for edge in data["link_values"][link_dict[1]]:
        edge_len = len(edge)
        #print(edge_len)
        relationship_type = edge[0]
        if relationship_type == link_dict[1] and edge[edge_len-1] == product_offering_id:
            warehouse_id = edge[edge_len-2]
          
            for node in data["node_values"][node_dict[1]]:
                node_len = len(node)
                #print(node_len)

            # Find the warehouse node attributes
            warehouse_attributes = next((node for node in data["node_values"][node_dict[1]] if node[node_len-1] == warehouse_id), None)
                                
            if warehouse_attributes:
                warehouses.append({
                    'warehouse_attributes': warehouse_attributes,
                    'edge_attributes': edge
                })
    
    return warehouses

# Query warehouses connected to the given product offering ID
file_path = '/home/rm_subhiksha/LAM_SRM/Query_jsongraph/NSS_2000_15_Simulation_timestamp_1.json'
product_offering_id = 'PO_001'
result = query_warehouses(product_offering_id, file_path)

# Print the result
if result:
    print(f"Product offering {product_offering_id} exists in the following warehouses:")
    for warehouse in result:
        print("Warehouse Attributes:", warehouse['warehouse_attributes'])
        print("Edge Attributes:", warehouse['edge_attributes'])
else:
    print(f"Product offering {product_offering_id} does not exist in any warehouse.")

Product offering PO_001 exists in the following warehouses:
Warehouse Attributes: ['WAREHOUSE', 'Warehouse_246', 'lam', 'New York', 'medium', 3055, 3055, 237, 10, 'W_246']
Edge Attributes: ['WAREHOUSEToPRODUCT_OFFERING', 342, 171.8551491996187, 'W_246', 'PO_001']
Warehouse Attributes: ['WAREHOUSE', 'Warehouse_249', 'lam', 'Arizona', 'small', 1533, 1533, 682, 5, 'W_249']
Edge Attributes: ['WAREHOUSEToPRODUCT_OFFERING', 625, 69.21480920409098, 'W_249', 'PO_001']
Warehouse Attributes: ['WAREHOUSE', 'Warehouse_287', 'lam', 'Georgia', 'small', 1360, 1360, 840, 5, 'W_287']
Edge Attributes: ['WAREHOUSEToPRODUCT_OFFERING', 131, 163.03411110684155, 'W_287', 'PO_001']
Warehouse Attributes: ['WAREHOUSE', 'Warehouse_298', 'lam', 'Florida', 'small', 1516, 1516, 332, 5, 'W_298']
Edge Attributes: ['WAREHOUSEToPRODUCT_OFFERING', 63, 14.379951834563807, 'W_298', 'PO_001']
Warehouse Attributes: ['WAREHOUSE', 'Warehouse_300', 'lam', 'Texas', 'medium', 3831, 3831, 820, 10, 'W_300']
Edge Attributes: ['WARE

# Warehouse : Inventory Level - Total no of product offerings in the warehouse etc, Storage cost - Cost of 1 unit of product offering etc

# Cost logic is for single unit and its applied everywhere cost occurs.


In [ ]:
import json

# Function to query warehouses connected to a product offering
def query_warehouses(product_offering_id, units, file_path):

    # Load the data from the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)

    facilities = []
    warehouses = []
    warehouse_edge = []
    warehouse_found = False

    for node in data["node_values"]["WAREHOUSE"]:
        print(node[2])
        print(node)
        node_len = len(node)

    for edge in data["link_values"]["WAREHOUSEToPRODUCT_OFFERING"]:
        edge_len = len(edge)
        relationship_type = edge[0]
        if relationship_type == "WAREHOUSEToPRODUCT_OFFERING" and edge[edge_len-1] == product_offering_id:
            warehouse_id = edge[edge_len-2]
            inventory_level = edge[edge_len-4]  # Adjust this index based on your data structure
            if inventory_level >= units:
                warehouse_edge.append({
                    'edge_attributes': edge
                })
                warehouse_found = True
                
                for node in data["node_values"]["WAREHOUSE"]:
                    node_len = len(node)
                    warehouse_type = node[2]
                    print(f"Warehouse type: {warehouse_type}")        

                print(f"The Product offering {product_offering_id} with demand of {units} units is available in the warehouse {warehouse_id} with inventory level {inventory_level}: warehouse type {warehouse_type}")
            else:
                print(f"The Product offering {product_offering_id} with demand of {units} units is not available in the warehouse {warehouse_id} : inventory level {inventory_level}")
                      
            # Find the warehouse node attributes
            warehouse_attributes = next((node for node in data["node_values"]["WAREHOUSE"] if node[node_len-1] == warehouse_id), None)

            if warehouse_attributes:
                warehouses.append({
                    'warehouse_attributes': warehouse_attributes,
                    'edge_attributes': edge
                })
    
    if warehouse_found:
        return warehouses, warehouse_edge
    else:
        # If no warehouse meets the requirement, check facilities
        for edge in data["link_values"]["FACILITYToPRODUCT_OFFERING"]:
            edge_len = len(edge)
            relationship_type = edge[0]
            if relationship_type == "FACILITYToPRODUCT_OFFERING" and edge[edge_len-1] == product_offering_id:
                facility_id = edge[edge_len-2]
              
                for node in data["node_values"]["FACILITY"]:
                    node_len = len(node)
            
                facility_attributes = next((node for node in data["node_values"]["FACILITY"] if node[node_len-1] == facility_id), None)        
                
                if facility_attributes:
                    facilities.append({
                        'facility_attributes': facility_attributes,
                        'edge_attributes': edge
                    })
    
    return facilities, warehouse_edge

# Query warehouses connected to the given product offering ID
file_path = '/home/rm_subhiksha/LAM_SRM/Query_jsongraph/NSS_2000_15_Simulation_timestamp_1.json'
product_offering_id = 'PO_001'
units = 800  # Example units required
result = query_warehouses(product_offering_id, units, file_path)

# Print the result
if result:
    warehouses, warehouse_edge = result
    if warehouses:
        print(f"Product offering {product_offering_id} exists in the following warehouse(s):")
        for entry in warehouses:
            print("Warehouse Attributes:", entry['warehouse_attributes'])
            print("Edge Attributes:", entry['edge_attributes'])
    if warehouse_edge:
        print("Warehouse Edge Attributes:")
        for entry in warehouse_edge:
            print(entry['edge_attributes'])
    if not warehouses:
        print(f"Product offering {product_offering_id} does not exist in any warehouse with the required units. Checking facilities...")
        for entry in facilities:
            print("Facility Attributes:", entry['facility_attributes'])
            print("Edge Attributes:", entry['edge_attributes'])
else:
    print(f"Product offering {product_offering_id} does not exist in any facility or warehouse with the required units.")


supplier
['WAREHOUSE', 'Warehouse_1', 'supplier', 'Washington', 'small', 1691, 1691, 924, 5, 'W_001']
supplier
['WAREHOUSE', 'Warehouse_2', 'supplier', 'Oregon', 'large', 8338, 8338, 54, 15, 'W_002']
supplier
['WAREHOUSE', 'Warehouse_3', 'supplier', 'New York', 'small', 2218, 2218, 476, 5, 'W_003']
supplier
['WAREHOUSE', 'Warehouse_4', 'supplier', 'Massachusetts', 'medium', 4837, 4059, 619, 10, 'W_004']
supplier
['WAREHOUSE', 'Warehouse_5', 'supplier', 'Massachusetts', 'medium', 4034, 4034, 657, 10, 'W_005']
supplier
['WAREHOUSE', 'Warehouse_6', 'supplier', 'New York', 'large', 6322, 6322, 661, 15, 'W_006']
supplier
['WAREHOUSE', 'Warehouse_7', 'supplier', 'Washington', 'large', 6684, 6684, 243, 15, 'W_007']
supplier
['WAREHOUSE', 'Warehouse_8', 'supplier', 'Massachusetts', 'large', 6887, 6887, 711, 15, 'W_008']
supplier
['WAREHOUSE', 'Warehouse_9', 'supplier', 'Washington', 'medium', 5153, 5153, 240, 10, 'W_009']
supplier
['WAREHOUSE', 'Warehouse_10', 'supplier', 'Georgia', 'small', 1

KeyError: 'warehouse_attributes'